In [41]:
import invisible_cities.reset.utils as rst_util
import numpy as np
from invisible_cities.core.system_of_units import pes, mm, mus, ns
import invisible_cities.reco.corrections    as corrf
import invisible_cities.database.load_db as dbf 

# Initialize RESET
iterations = 100
pitch      = 10
run_number = 4495
nsipms     = 1792
npmts      = 1 
dist       = 20. 
sipm_dist  = 20. 
pmt_dist   = 200 # max distance included in the param file
sipm_thr   = 5.
#x_size     = 2.
#y_size     = 2.

x_size     = 10.
y_size     = 10.

rmax       = 198 #
slice_width = 2.
pmt_param  = "/home/jmbenlloch/reset_data/mapas/PMT_Map_corr_keV.h5"
sipm_param = "/home/jmbenlloch/reset_data/mapas/SiPM_Map_corr_z0.0_keV.h5"

pmap_file = '4495_21215.h5'

pmap_conf = {}
pmap_conf['s1_emin'] = 54 * pes 
pmap_conf['s1_emax'] = 2400. * pes 
pmap_conf['s1_wmin'] = 7*25 * ns
pmap_conf['s1_wmax'] = 16*25 * ns
pmap_conf['s1_hmin'] = 12. * pes 
pmap_conf['s1_hmax'] = 400. * pes 
pmap_conf['s1_num'] = 1 
pmap_conf['s1_ethr'] = 0.5 * pes 

pmap_conf['s2_emin'] = 200000. * pes 
pmap_conf['s2_emax'] = 2.e6 * pes 
pmap_conf['s2_wmin'] = 4.* mus 
pmap_conf['s2_wmax'] = 500. * mus 
pmap_conf['s2_hmin'] = 600. * pes 
pmap_conf['s2_hmax'] = 60000. * pes 
pmap_conf['s2_numMin'] = 1 
pmap_conf['s2_numMax'] = 3 
pmap_conf['s2_ethr'] = 0. * pes 
pmap_conf['s2_nsipmmin'] = 1 
pmap_conf['s2_nsipmmax'] = 1792

# Read file and select peaks
selector = rst_util.refresh_selector(pmap_conf)
s1s, s2s, s2sis, peaks = rst_util.load_and_select_peaks(pmap_file, 21215, selector)

# Lifetime correction
ZCorr = corrf.LifetimeCorrection(1093.77, 23.99)

# Sensors info
data_sipm = dbf.DataSiPM(run_number)

# Take the first peak
peak = next(iter(peaks))
evt = 21215
sipm_thr = sipm_thr

# Prepare data
voxels_data, slices, energies, zs = rst_util.prepare_data(s1s, s2s, s2sis, slice_width, evt, peak, data_sipm, nsipms, sipm_thr, dist, ZCorr)
slices_start = rst_util.slices_start(voxels_data, x_size, y_size)

In [42]:
import invisible_cities.reset.reset_serial as rst_serial

# Create voxels

In [43]:
rst_voxels = rst_serial.create_voxels(voxels_data, slices_start, x_size, y_size, rmax)

In [44]:
rst_voxels

ResetVoxels(nslices=6, nvoxels=1717, voxels=array([(115., 25.,  6.3964443), (115., 35.,  6.3964443),
       (115., 45.,  6.3964443), ..., (195.,  5., 10.671945 ),
       (195., 15., 10.671945 ), (195., 25., 10.671945 )],
      dtype=[('x', '<f4'), ('y', '<f4'), ('E', '<f4')]), slice_ids=array([0, 0, 0, ..., 5, 5, 5], dtype=int32), slice_start=array([   0,   67,  173,  470,  872, 1404, 1717], dtype=int32), address=array([   0,    1,    2, ..., 1717, 1717, 1717]))

# Anode response

In [15]:
nslices = voxels_data.xmin.shape[0]
anode_response = rst_serial.create_anode_response(slices)

# Create cathode response

In [16]:
cath_response = energies

# Compute probs

In [17]:
import math

In [18]:
sipm_param_file  = "/home/jmbenlloch/reset_data/mapas/SiPM_Map_corr_z0.0_keV.h5"
sipm_params = rst_util.read_corrections_file(sipm_param_file, 'SiPM')

sipms_per_voxel = int(math.floor(2 * sipm_dist / pitch) + 1)**2
voxels_per_sipm = int((2 * sipm_dist)**2 / (x_size * y_size))
xs = data_sipm.X
ys = data_sipm.Y

In [ ]:
sipm_probs = rst_serial.compute_probabilities_cuda(rst_voxels, xs, ys, nsipms, sipms_per_voxel, sipm_dist, sipm_params, anode_response)

In [ ]:
sipm_probs.nprobs

### PMTs

In [19]:
pmt_param_file = "/home/jmbenlloch/reset_data/mapas/PMT_Map_corr_keV.h5"
pmt_params = rst_util.read_corrections_file(pmt_param_file, 'PMT')

pmts_per_voxel = npmts
voxels_per_pmt = int((2 * pmt_dist)**2 / (x_size * y_size))

xs_pmts = np.array([0.], dtype='f4')
ys_pmts = np.array([0.], dtype='f4')

In [20]:
pmt_probs = rst_serial.compute_probabilities_cuda(rst_voxels, xs_pmts, ys_pmts, npmts, pmts_per_voxel, pmt_dist, pmt_params, cath_response)

# Sensor probs

In [ ]:
sipm_sns_probs = rst_serial.compute_sensor_probs(sipm_probs, nslices, nsipms, rst_voxels.slice_ids)
pmt_sns_probs = rst_serial.compute_sensor_probs(pmt_probs, nslices, npmts, rst_voxels.slice_ids)

# Forward denom

In [ ]:
sipm_fwd_denom = rst_serial.forward_denoms(nsipms, nslices, rst_voxels.voxels, sipm_sns_probs)
pmt_fwd_denom = rst_serial.forward_denoms(npmts, nslices, rst_voxels.voxels, pmt_sns_probs)

# MLEM step

In [ ]:
rst_serial.compute_mlem(iterations, rst_voxels, nsipms, sipm_probs, sipm_sns_probs, npmts, pmt_probs, pmt_sns_probs)

In [ ]:
rst_voxels

# RESET serial

In [9]:
rst_voxels = rst_serial.create_voxels(voxels_data, slices_start, x_size, y_size, rmax)
anode_response = rst_serial.create_anode_response(slices)
cath_response = energies

sipm_param_file  = "/home/jmbenlloch/reset_data/mapas/SiPM_Map_corr_z0.0_keV.h5"
sipm_params = rst_util.read_corrections_file(sipm_param_file, 'SiPM')

#sipms_per_voxel = int(math.floor(2 * sipm_dist / pitch) + 1)**2
#voxels_per_sipm = int((2 * sipm_dist)**2 / (x_size * y_size))

sipm_ratios = rst_util.compute_sipm_ratio(sipm_dist, pitch, x_size, y_size)
xs = data_sipm.X
ys = data_sipm.Y

sipm_probs = rst_serial.compute_probabilities_cuda(rst_voxels, xs, ys, nsipms, sipm_ratios.sns_per_voxel, sipm_dist, sipm_params, anode_response)

pmt_param_file = "/home/jmbenlloch/reset_data/mapas/PMT_Map_corr_keV.h5"
pmt_params = rst_util.read_corrections_file(pmt_param_file, 'PMT')

#pmts_per_voxel = npmts
#voxels_per_pmt = int((2 * pmt_dist)**2 / (x_size * y_size))
pmt_ratios = rst_util.compute_pmt_ratio(pmt_dist, npmts, x_size, y_size)

xs_pmts = np.array([0.], dtype='f4')
ys_pmts = np.array([0.], dtype='f4')

pmt_probs = rst_serial.compute_probabilities_cuda(rst_voxels, xs_pmts, ys_pmts, npmts, pmt_ratios.sns_per_voxel, pmt_dist, pmt_params, cath_response)

sipm_sns_probs = rst_serial.compute_sensor_probs(sipm_probs, nslices, nsipms, rst_voxels.slice_ids)
pmt_sns_probs = rst_serial.compute_sensor_probs(pmt_probs, nslices, npmts, rst_voxels.slice_ids)

sipm_fwd_denom = rst_serial.forward_denoms(nsipms, nslices, rst_voxels.voxels, sipm_sns_probs)
pmt_fwd_denom = rst_serial.forward_denoms(npmts, nslices, rst_voxels.voxels, pmt_sns_probs)

rst_serial.compute_mlem(iterations, rst_voxels, nsipms, sipm_probs, sipm_sns_probs, npmts, pmt_probs, pmt_sns_probs)

NameError: name 'nslices' is not defined

# RESET CUDA

In [21]:
import invisible_cities.reset.reset_functions_event as rstf
import invisible_cities.reset.memory as rst_mem

In [22]:
rst = rstf.RESET(run_number, nsipms, npmts, dist, sipm_dist, pmt_dist, sipm_thr, x_size, y_size, rmax, sipm_param, pmt_param)

## Voxels

In [23]:
import pycuda.driver as cuda
voxels_data_d = rst_mem.copy_voxels_data_h2d(voxels_data)
slices_data_d = rst_mem.copy_slice_data_h2d(slices)

slices_start_nc_d = cuda.to_device(slices_start)

In [24]:
rst_voxels_d, slice_ids_h = rstf.create_voxels(rst.cudaf, voxels_data_d, rst.xsize,
                                            rst.ysize, rst.rmax, rst.max_voxels,
                                            slices_start_nc_d, int(slices_start[-1]))

rst_voxels_h = rst_mem.copy_voxels_d2h(rst_voxels_d)

In [25]:
rst_voxels_h

ResetVoxels(nslices=6, nvoxels=1717, voxels=array([(115., 25.,  6.3964443), (115., 35.,  6.3964443),
       (115., 45.,  6.3964443), ..., (195.,  5., 10.671945 ),
       (195., 15., 10.671945 ), (195., 25., 10.671945 )],
      dtype=[('x', '<f4'), ('y', '<f4'), ('E', '<f4')]), slice_ids=array([0, 0, 0, ..., 5, 5, 5], dtype=int32), slice_start=array([   0,   67,  173,  470,  872, 1404, 1717], dtype=int32), address=array([   0,    1,    2, ..., 1717, 1717, 1717], dtype=int32))

## Anode response

In [26]:
anode_d = rstf.create_anode_response(rst.cudaf, slices_data_d)

total_sensors = nsipms * voxels_data_d.nslices
anode_h = cuda.from_device(anode_d, (total_sensors,), np.dtype('f4'))

## Cath response

In [27]:
cath_d  = rstf.create_cath_response(energies)

## Compute probs

In [28]:
sipm_probs_d = rstf.compute_probabilites(rst.cudaf,
                    rst_voxels_d, rst.nsipms, sipms_per_voxel,
                    rst.sipm_dist, rst.xs_sipms_d,
                    rst.ys_sipms_d, rst.sipm_param,
                    slices_start_nc_d,
                    rst.xsize, rst.ysize, anode_d)


sipm_probs_h = rst_mem.copy_probs_d2h(sipm_probs_d, rst_voxels.nvoxels, nslices, nsipms)

In [29]:
pmt_probs_d = rstf.compute_probabilites(rst.cudaf,
                    rst_voxels_d, rst.npmts, pmts_per_voxel,
                    rst.pmt_dist, rst.xs_pmts_d,
                    rst.ys_pmts_d, rst.pmt_param,
                    slices_start_nc_d,
                    rst.xsize, rst.ysize, cath_d)

pmt_probs_h = rst_mem.copy_probs_d2h(pmt_probs_d, rst_voxels.nvoxels, nslices, npmts)

## Compute Sensor probs

In [34]:
sipm_sns_probs_d = rstf.compute_sensor_probs(rst.cudaf,
                  rst_voxels_d, rst.nsipms,
                  voxels_per_sipm, rst.sipm_dist, rst.xs_sipms_d,
                  rst.ys_sipms_d, rst.sipm_param,
                  slices_start_nc_d, voxels_data_d,
                  rst.xsize, rst.ysize,
                  sipm_probs_d.sensor_start)

In [33]:
pmt_sns_probs_d = rstf.compute_sensor_probs(rst.cudaf,
                  rst_voxels_d, rst.npmts,
                  voxels_per_pmt, rst.pmt_dist, rst.xs_pmts_d,
                  rst.ys_pmts_d, rst.pmt_param,
                  slices_start_nc_d, voxels_data_d,
                  rst.xsize, rst.ysize,
                  pmt_probs_d.sensor_start)

## Forward denom

In [ ]:
import pycuda
sipm_denoms   = pycuda.gpuarray.zeros(int(nsipms * voxels_data_d.nslices), np.dtype('f4'))
sipm_denoms_d = sipm_denoms.gpudata

block_sipm = 1024
grid_sipm  = math.ceil(sipm_sns_probs_d.nsensors / block_sipm)

cuda_forward_denom = rst.cudaf.get_function('forward_denom')

cuda_forward_denom(sipm_denoms_d, sipm_sns_probs_d.sensor_start,
                    sipm_sns_probs_d.sensor_start_ids, sipm_sns_probs_d.probs,
                    sipm_sns_probs_d.voxel_ids, rst_voxels_d.voxels, 
                    sipm_sns_probs_d.nsensors,
                    block=(block_sipm, 1, 1), grid=(grid_sipm, 1))

sipm_denoms_h = cuda.from_device(sipm_denoms_d, (rst.nsipms * nslices,), np.dtype('f4'))

## Compute MLEM

In [ ]:
voxels_h = rstf.compute_mlem(rst.cudaf, rst_voxels_d, voxels_data_d.nslices,
          rst.npmts,  pmt_probs_d,  pmt_sns_probs_d,
          rst.nsipms, sipm_probs_d, sipm_sns_probs_d)

In [ ]:
voxels_h.shape

In [ ]:
rst_voxels.voxels['x']

In [ ]:
np.allclose(voxels_h['x'], rst_voxels.voxels['x'])
np.allclose(voxels_h['y'], rst_voxels.voxels['y'])
np.allclose(voxels_h['E'], rst_voxels.voxels['E'])

# CUDA version

In [10]:
import invisible_cities.reset.reset_functions_event as rstf
import invisible_cities.reset.memory as rst_mem
import pycuda.driver as cuda

rst = rstf.RESET(run_number, nsipms, npmts, dist, sipm_dist, pmt_dist, 
                 sipm_thr, x_size, y_size, rmax, sipm_param, pmt_param)

voxels_data_d = rst_mem.copy_voxels_data_h2d(voxels_data)
slices_data_d = rst_mem.copy_slice_data_h2d(slices)

slices_start_nc_d = cuda.to_device(slices_start)

rst_voxels_d, slice_ids_h = rstf.create_voxels(rst.cudaf, voxels_data_d, rst.xsize,
                                            rst.ysize, rst.rmax, rst.max_voxels,
                                            slices_start_nc_d, int(slices_start[-1]))

rst_voxels_h = rst_mem.copy_voxels_d2h(rst_voxels_d)

anode_d = rstf.create_anode_response(rst.cudaf, slices_data_d)

total_sensors = nsipms * voxels_data_d.nslices
anode_h = cuda.from_device(anode_d, (total_sensors,), np.dtype('f4'))

cath_d  = rstf.create_cath_response(energies)

sipm_probs_d = rstf.compute_probabilites(rst.cudaf,
                    rst_voxels_d, rst.nsipms, sipms_per_voxel,
                    rst.sipm_dist, rst.xs_sipms_d,
                    rst.ys_sipms_d, rst.sipm_param,
                    slices_start_nc_d,
                    rst.xsize, rst.ysize, anode_d)


sipm_probs_h = rst_mem.copy_probs_d2h(sipm_probs_d, rst_voxels.nvoxels, nslices, nsipms)

pmt_probs_d = rstf.compute_probabilites(rst.cudaf,
                    rst_voxels_d, rst.npmts, pmts_per_voxel,
                    rst.pmt_dist, rst.xs_pmts_d,
                    rst.ys_pmts_d, rst.pmt_param,
                    slices_start_nc_d,
                    rst.xsize, rst.ysize, cath_d)

pmt_probs_h = rst_mem.copy_probs_d2h(pmt_probs_d, rst_voxels.nvoxels, nslices, npmts)

sipm_sns_probs_d = rstf.compute_sensor_probs(rst.cudaf,
                  rst_voxels_d, rst.nsipms, sipms_per_voxel,
                  voxels_per_sipm, rst.sipm_dist, rst.xs_sipms_d,
                  rst.ys_sipms_d, rst.sipm_param,
                  slices_start_nc_d, voxels_data_d,
                  rst.xsize, rst.ysize,
                  sipm_probs_d.sensor_start)

pmt_sns_probs_d = rstf.compute_sensor_probs(rst.cudaf,
                  rst_voxels_d, rst.npmts, pmts_per_voxel,
                  voxels_per_pmt, rst.pmt_dist, rst.xs_pmts_d,
                  rst.ys_pmts_d, rst.pmt_param,
                  slices_start_nc_d, voxels_data_d,
                  rst.xsize, rst.ysize,
                  pmt_probs_d.sensor_start)

sipm_denoms   = pycuda.gpuarray.zeros(int(nsipms * voxels_data_d.nslices), np.dtype('f4'))
sipm_denoms_d = sipm_denoms.gpudata

block_sipm = 1024
grid_sipm  = math.ceil(sipm_sns_probs_d.nsensors / block_sipm)

cuda_forward_denom = rst.cudaf.get_function('forward_denom')

cuda_forward_denom(sipm_denoms_d, sipm_sns_probs_d.sensor_start,
                    sipm_sns_probs_d.sensor_start_ids, sipm_sns_probs_d.probs,
                    sipm_sns_probs_d.voxel_ids, rst_voxels_d.voxels, 
                    sipm_sns_probs_d.nsensors,
                    block=(block_sipm, 1, 1), grid=(grid_sipm, 1))

sipm_denoms_h = cuda.from_device(sipm_denoms_d, (rst.nsipms * nslices,), np.dtype('f4'))

voxels_h = rstf.compute_mlem(rst.cudaf, rst_voxels_d, voxels_data_d.nslices,
          rst.npmts,  pmt_probs_d,  pmt_sns_probs_d,
          rst.nsipms, sipm_probs_d, sipm_sns_probs_d)

NameError: name 'sipms_per_voxel' is not defined

In [48]:
'ResetVoxels'    , 'nslices nvoxels voxels slice_ids slice_start address'),

SyntaxError: invalid syntax (<ipython-input-48-1f396c4bc9e1>, line 1)

In [49]:
from invisible_cities.evm.ic_containers import ResetVoxels

In [51]:
ResetVoxels(rst_voxels.nslices,
            rst_voxels.nvoxels,
            rst_voxels.voxels.copy(),
            rst_voxels.slice_ids,
            rst_voxels.slice_start,
            rst_voxels.address)

ResetVoxels(nslices=6, nvoxels=1717, voxels=array([(115., 25.,  6.3964443), (115., 35.,  6.3964443),
       (115., 45.,  6.3964443), ..., (195.,  5., 10.671945 ),
       (195., 15., 10.671945 ), (195., 25., 10.671945 )],
      dtype=[('x', '<f4'), ('y', '<f4'), ('E', '<f4')]), slice_ids=array([0, 0, 0, ..., 5, 5, 5], dtype=int32), slice_start=array([   0,   67,  173,  470,  872, 1404, 1717], dtype=int32), address=array([   0,    1,    2, ..., 1717, 1717, 1717]))